# Interacting with Campaigns <a class="anchor" id="top"></a>

In this notebook, you will deploy and interact with campaigns in Amazon Personalize.

1. [Introduction](#intro)
1. [Create campaigns](#create)
1. [Interact with campaigns](#interact)
1. [Batch recommendations](#batch)
1. [Wrap up](#wrapup)

## Introduction <a class="anchor" id="intro"></a>
[Back to top](#top)

At this point, you should have several solutions and at least one solution version for each. Once a solution version is created, it is possible to get recommendations from them, and to get a feel for their overall behavior.

This notebook starts off by deploying each of the solution versions from the previous notebook into individual campaigns. Once they are active, there are resources for querying the recommendations, and helper functions to digest the output into something more human-readable. 

As you with your customer on Amazon Personalize, you can modify the helper functions to fit the structure of their data input files to keep the additional rendering working.

To get started, once again, we need to import libraries, load values from previous notebooks, and load the SDK.

In [89]:
import time
#import datetime
from time import sleep
import json
from datetime import datetime
import uuid
import random

import boto3, botocore
import pandas as pd
import numpy as np

In [90]:
%store -r

In [91]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's event streaming
personalize_events = boto3.client(service_name='personalize-events')

## Add New items to Catalog <a class="anchor" id="interact"></a>
[Back to top](#top)

Now that all campaigns are deployed and active, we will need to add content to our catalog as it is released. Given that this content is completely new, it will not have interactions in the dataset for the user personalization solution to train on. Due to this we will use the exploration capability in the user personalization campaign. 

First we will get the current time, which we will then use as the CREATION_TIMESTAMP for our new items. Amazon Personalize uses creation timestamp data (in UNIX epoch time format, in seconds) to calculate the age of an item and adjust recommendations accordingly.

If creation timestamp data is missing for one or more items, Amazon Personalize infers this information from interaction data, if any, and uses the timestamp of the item’s oldest interaction data as the item's creation date. If an item has no interaction data, its creation date is set as the timestamp of the latest interaction in the training set and is considered a new item. 

The creation timestamp data will be added to the user personalization model (as long as exploration value is above 0 and the creation timestamp is within the exploration_item_age_cut_off.

In [11]:
#original_data = pd.read_csv(dataset_dir + '/movies.csv', sep=',', encoding='latin-1', dtype={'movieId': "int64", 'title': "str"})
#original_data = pd.read_csv(dataset_dir + '/movies.csv')
original_data = pd.read_csv(dataset_dir + '/movies.csv', sep=',', usecols=[0,1,2], encoding='latin-1', dtype={'movieId': "object", 'title': "str", 'genres': 'str'},index_col=0)

original_data.tail(5)

,title,genres
movieId,,
193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
193585,Flint (2017),Drama
193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation
193609,Andrew Dice Clay: Dice Rules (1991),Comedy


In [92]:
currenttime = round(time.time()) - 1000
currentyear = datetime.utcfromtimestamp(currenttime).strftime('%Y')

In [93]:
new_titles = {'ITEM_ID': [itemmetadata_df.last_valid_index() + 1, itemmetadata_df.last_valid_index() + 2, itemmetadata_df.last_valid_index() + 3, itemmetadata_df.last_valid_index() + 4], 'title': ['New Title 1' + " (" + str(currentyear) + ")", 'New Title 2' + " (" + str(currentyear) + ")", 'New Title 3' + " (" + str(currentyear) + ")", 'New Title 4' + " (" + str(currentyear) + ")"],'GENRE': ['Action|Comedy', 'Sci-Fi|Fantasy', 'Drama|Thriller', 'Documentary|IMAX'],'YEAR': [currentyear, currentyear, currentyear, currentyear],'CREATION_TIMESTAMP': [round(time.time()) - 1000, round(time.time()) - 2000, round(time.time()) - 3000, round(time.time())]}
new_titles_df = pd.DataFrame(new_titles)
new_titles_df=new_titles_df.astype(dtype= {"ITEM_ID":"int64", "title":"object", "GENRE":"object","YEAR":"object","CREATION_TIMESTAMP":"int64"})
new_titles_df=new_titles_df.set_index('ITEM_ID')
new_titles_df

,title,GENRE,YEAR,CREATION_TIMESTAMP
ITEM_ID,,,,
193610,New Title 1 (2021),Action|Comedy,2021,1619813039
193611,New Title 2 (2021),Sci-Fi|Fantasy,2021,1619812039
193612,New Title 3 (2021),Drama|Thriller,2021,1619811039
193613,New Title 4 (2021),Documentary|IMAX,2021,1619814039


In [94]:
items_df

,title
movieId,
1,Toy Story (1995)
2,Jumanji (1995)
3,Grumpier Old Men (1995)
4,Waiting to Exhale (1995)
5,Father of the Bride Part II (1995)
...,...
193581,Black Butler: Book of the Atlantic (2017)
193583,No Game No Life: Zero (2017)
193585,Flint (2017)


In [96]:
itemmetadata_df = itemmetadata_df.append(new_titles_df, ignore_index=False)
itemmetadata_df = itemmetadata_df[['GENRE','YEAR','CREATION_TIMESTAMP']]
itemmetadata_df.tail(10)

,GENRE,YEAR,CREATION_TIMESTAMP
ITEM_ID,,,
193587,Action|Animation,2018,0
193609,Comedy,1991,0
193610,Action|Comedy,2021,1619813039
193611,Sci-Fi|Fantasy,2021,1619812039
193612,Drama|Thriller,2021,1619811039
193613,Documentary|IMAX,2021,1619814039
193610,Action|Comedy,2021,1619813039
193611,Sci-Fi|Fantasy,2021,1619812039
193612,Drama|Thriller,2021,1619811039


We will also add the new titles to the look up table from the previous notebook

In [97]:
new_items_df = new_titles_df[['title']]
new_items_df = new_items_df.rename_axis('movieId')
items_df = items_df.append(new_items_df, ignore_index=False)
items_df

,title
movieId,
1,Toy Story (1995)
2,Jumanji (1995)
3,Grumpier Old Men (1995)
4,Waiting to Exhale (1995)
5,Father of the Bride Part II (1995)
...,...
193609,Andrew Dice Clay: Dice Rules (1991)
193610,New Title 1 (2021)
193611,New Title 2 (2021)


now we will add the titles to item metadata via a batch import process. Note, you can also add these in realtime using the putitems api (see code example below) 

```python
def add_title(item_id, genre, year, creationtimestamp):
   
    itemId= str(item_id)
    #print(itemId)
    itemdata = {
        "genre": genre,
        "year": year,
        "creationTimestamp": creationtimestamp
    }
# Make Call
    personalize_events.put_items(
    datasetArn=items_dataset_arn,
    items=[
        {
            'itemId': str(itemId),
            'properties': json.dumps(itemdata)
        },
    ]
    )
```
and then execute by calling the function

```python
for index, row in new_titles_df.iterrows():
    print("Adding Item #" + str(index) + " " + row['title'] + " with genres " + row['genres'])
    add_title(item_id=index,genre=row['genres'],year=row['year'],creationtimestamp=row['CREATION_TIMESTAMP'])
```

Similar to 02_Validating_and_Importing_Item_Metadata.ipynb we will store the updated item_metadata_df to a csv file and upload it to S3

In [98]:
itemmetadata_filename = "item-meta.csv"
itemmetadata_df.to_csv((data_dir+"/"+itemmetadata_filename), index=True, float_format='%.0f')

In [99]:
itemmetadata_file_path = data_dir + "/" + itemmetadata_filename
boto3.Session().resource('s3').Bucket(bucket_name).Object(itemmetadata_filename).upload_file(itemmetadata_file_path)
interactions_s3DataPath = "s3://"+bucket_name+"/"+itemmetadata_filename

Once the updated file is in S3, we will start a dataset import job

In [100]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-poc-item-import" + str(currenttime),
    datasetArn = items_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, itemmetadata_filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:832194813872:dataset-import-job/personalize-poc-item-import1619813038",
  "ResponseMetadata": {
    "RequestId": "b8635922-ee01-4d14-b9e2-5f159e675b0d",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 30 Apr 2021 23:38:53 GMT",
      "x-amzn-requestid": "b8635922-ee01-4d14-b9e2-5f159e675b0d",
      "content-length": "125",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


We will check the status of the import.

In [101]:
%%time

max_time = time.time() + 6*60*60 # 6 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: CREATE PENDING
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE
CPU times: user 115 ms, sys: 1.34 ms, total: 117 ms
Wall time: 15min 1s


To update the model(solutionVersion), we ran the createSolutionVersion with trainingMode set to UPDATE. This updates the model with the latest item information and adjusts the exploration according to implicit feedback from the users. This is not equivalent to training a model, which you can do by setting trainingMode to FULL. You should perform full training less frequently, typically once every 1–5 days. When the new updated solutionVersion is created, you can update the campaign to get recommendations using it. NOTE: This happens automatically every 2 hours, for the purposes of this workshop we are running it on demand.

In [102]:
user_personalization_update_solution_response = personalize.create_solution_version(
    solutionArn = user_personalization_solution_arn, 
    trainingMode='UPDATE')
new_user_personalization_solution_version_arn = user_personalization_update_solution_response['solutionVersionArn']
print("Creating solution version: {}".format(new_user_personalization_solution_version_arn))


Creating solution version: arn:aws:personalize:us-east-1:832194813872:solution/personalize-poc-userpersonalization/061d7a5c


In [103]:
status = None
max_time = time.time() + 60*60 # 1 hour
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
    solutionVersionArn = new_user_personalization_solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))

    if status == "ACTIVE" or status == "CREATE FAILED":
         break
    time.sleep(60)

SolutionVersion: CREATE PENDING
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGR

Now that the solution is updated with the new solution version, including item-metadata information about our new titles, we can update the campaign to use this version.

In [104]:
userpersonalization_update_campaign_response = personalize.update_campaign(campaignArn=userpersonalization_campaign_arn, solutionVersionArn=new_user_personalization_solution_version_arn)
userpersonalization_campaign_arn = userpersonalization_update_campaign_response['campaignArn']
print(json.dumps(userpersonalization_update_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:us-east-1:832194813872:campaign/personalize-poc-userpersonalization",
  "ResponseMetadata": {
    "RequestId": "7c4cbf18-5cb9-4a3b-9808-0ddd4a7de80c",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sat, 01 May 2021 00:36:29 GMT",
      "x-amzn-requestid": "7c4cbf18-5cb9-4a3b-9808-0ddd4a7de80c",
      "content-length": "105",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


Now lets check the status of the solution version update.

In [105]:
describe_campaign_update_response = personalize.describe_campaign(
            campaignArn = userpersonalization_campaign_arn)
status = describe_campaign_update_response["campaign"]["status"]
print(status)

ACTIVE


Note that the campaign itself is ACTIVE, however the latestCampaignUpdate is either in CREATE PENDING or CREATE IN_PROGRESS, the campaign will remain active with the previous solution version until the new solution version is active at which point it will become active in the current campaign. We will watch for the latestCampaignUpdate to become ACTIVE. Our new items will become available once the campaign is active with the new solution version. 


In [106]:
status = None
max_time = time.time() + 60*60 # 1 hour
while time.time() < max_time:
    describe_campaign_update_response = personalize.describe_campaign(
            campaignArn = userpersonalization_campaign_arn
        )
    status = describe_campaign_update_response["campaign"]["latestCampaignUpdate"]["status"]
    print("CampaignUpdate: {}".format(status))

    if status == "ACTIVE" or status == "CREATE FAILED":
         break
    time.sleep(60)

CampaignUpdate: CREATE IN_PROGRESS
CampaignUpdate: CREATE IN_PROGRESS
CampaignUpdate: CREATE IN_PROGRESS
CampaignUpdate: CREATE IN_PROGRESS
CampaignUpdate: CREATE IN_PROGRESS
CampaignUpdate: CREATE IN_PROGRESS
CampaignUpdate: CREATE IN_PROGRESS
CampaignUpdate: CREATE IN_PROGRESS
CampaignUpdate: CREATE IN_PROGRESS
CampaignUpdate: CREATE IN_PROGRESS
CampaignUpdate: CREATE IN_PROGRESS
CampaignUpdate: CREATE IN_PROGRESS
CampaignUpdate: CREATE IN_PROGRESS
CampaignUpdate: ACTIVE


We will create a movie lookup function as in the '05_Interacting_with_Campaigns_and_Filters.ipynb' notebook

In [107]:
def get_movie_by_id(movie_id, movie_df=items_df):
    """
    This takes in an movie_id from Personalize so it will be a string,
    converts it to an int, and then does a lookup in a default or specified
    dataframe.
    
    A really broad try/except clause was added in case anything goes wrong.
    
    Feel free to add more debugging or filtering here to improve results if
    you hit an error.
    """
    try:
        return movie_df.loc[int(movie_id)]['title']
    except:
        return "Error obtaining title"

Now let's test a few simple values to check our error catching.

In [108]:
# A known good id (NewTitle4)
print(get_movie_by_id(movie_id=str(items_df.last_valid_index())))
# A bad type of value
print(get_movie_by_id(movie_id='987.9393939'))
# Really bad values
print(get_movie_by_id(movie_id='Steve'))

New Title 4 (2021)
Error obtaining title
Error obtaining title


Great! Now we have a way of rendering results. 

In [109]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df(recommendations_df, movie_ID):
    # Get the movie name
    movie_name = get_movie_by_id(movie_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = sims_campaign_arn,
        itemId = str(movie_ID),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [movie_name])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

### User Personalization

HRNN is one of the more advanced algorithms provided by Amazon Personalize, and is the underlying algorithm in the user personalization recipe. It supports personalization of the items for a specific user based on their past behavior and can intake real time events in order to alter recommendations for a user without retraining. 

Since HRNN relies on having a sampling of users, let's load the data we need for that and select 3 random users. Since Movielens does not include user data, we will select 3 random numbers from the range of user id's in the dataset.

In [110]:
if not USE_FULL_MOVIELENS:
    users = random.sample(range(1, 600), 10)
else:
    users = random.sample(range(1, 162000), 10)
users

[417, 213, 558, 236, 168, 22, 480, 331, 339, 198]

Now we render the recommendations for our 3 random users from above. After that, we will explore real-time interactions before moving on to Personalized Ranking.

Again, we create a helper function to render the results in a nice dataframe.

#### API call results

In [111]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df_users(recommendations_df, user_id):
    # Get the movie name
    #movie_name = get_movie_by_id(artist_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    #print(recommendation_list)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [user_id])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

In [112]:
recommendations_df_users = pd.DataFrame()
#users = users_df.sample(3).index.tolist()

for user in users:
    recommendations_df_users = get_new_recommendations_df_users(recommendations_df_users, user)

recommendations_df_users

,417,213,558,236,168,22,480,331,339,198
0,New Title 3 (2021),New Title 3 (2021),New Title 1 (2021),New Title 1 (2021),New Title 3 (2021),New Title 1 (2021),New Title 3 (2021),New Title 3 (2021),New Title 3 (2021),New Title 1 (2021)
1,New Title 1 (2021),New Title 1 (2021),New Title 3 (2021),Payback (1999),New Title 1 (2021),New Title 3 (2021),New Title 1 (2021),New Title 1 (2021),New Title 1 (2021),New Title 3 (2021)
2,New Title 4 (2021),Dr. Strangelove or: How I Learned to Stop Worr...,American Pie (1999),New Title 3 (2021),Chinatown (1974),Meet the Parents (2000),Bambi (1942),New Title 4 (2021),New Title 4 (2021),Raising Arizona (1987)
3,Shutter Island (2010),American Beauty (1999),There's Something About Mary (1998),Go (1999),Apocalypse Now (1979),Superbad (2007),Dumbo (1941),New Title 2 (2021),New Title 2 (2021),Spaceballs (1987)
4,District 9 (2009),"Matrix, The (1999)","Wedding Singer, The (1998)",Gone in 60 Seconds (2000),Rear Window (1954),"Lord of the Rings: The Two Towers, The (2002)",How the Grinch Stole Christmas! (1966),Logan (2017),Interstellar (2014),Predator (1987)
5,"Hobbit: The Desolation of Smaug, The (2013)",Casablanca (1942),Emma (1996),Air Force One (1997),"Clockwork Orange, A (1971)",School of Rock (2003),To Kill a Mockingbird (1962),Gone Girl (2014),Birdman: Or (The Unexpected Virtue of Ignoranc...,Ferris Bueller's Day Off (1986)
6,"Hobbit: An Unexpected Journey, The (2012)","Godfather, The (1972)",Austin Powers: The Spy Who Shagged Me (1999),Charlie's Angels (2000),This Is Spinal Tap (1984),"Hangover, The (2009)",Peter Pan (1953),Inglourious Basterds (2009),Whiplash (2014),Heathers (1989)
7,Harry Potter and the Goblet of Fire (2005),Eternal Sunshine of the Spotless Mind (2004),Runaway Bride (1999),Blade II (2002),Rashomon (RashÃ´mon) (1950),Step Brothers (2008),Citizen Kane (1941),Deadpool 2 (2018),"Big Short, The (2015)",Cocoon (1985)
8,Django Unchained (2012),"Lord of the Rings: The Fellowship of the Ring,...",Grease (1978),Mission: Impossible II (2000),New Title 4 (2021),Borat: Cultural Learnings of America for Make ...,Fantasia (1940),The Imitation Game (2014),The Revenant (2015),"Untouchables, The (1987)"
9,Argo (2012),Forrest Gump (1994),William Shakespeare's Romeo + Juliet (1996),From Dusk Till Dawn (1996),Scarface (1983),Bruce Almighty (2003),Cinderella (1950),John Wick (2014),Gone Girl (2014),RoboCop (1987)


Here we can see the cold items are being recommended based on the exploration weight set when we created the campaign.Now lets send in some interactions from the sample users to simulate viewers engaging with this cold start content.


In [117]:
session_dict = {}

def send_movie_click(USER_ID, ITEM_ID, EVENT_TYPE):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        session_ID = session_dict[str(USER_ID)]
    except:
        session_dict[str(USER_ID)] = str(uuid.uuid1())
        session_ID = session_dict[str(USER_ID)]
        
    # Configure Properties:
    event = {
    "itemId": str(ITEM_ID),
    }
    event_json = json.dumps(event)
        
    # Make Call
    
    response = personalize_events.put_events(
    trackingId = TRACKING_ID,
    userId= str(USER_ID),
    sessionId = session_ID,
    eventList = [{
        'sentAt': int(time.time()),
        'eventType': str(EVENT_TYPE),
        'properties': event_json
        }]
    )
   
def get_new_recommendations_df_users_real_time(recommendations_df, user_id, item_id, event_type):
    # Get the artist name (header of column)
    movie_name = get_movie_by_id(item_id)
    # Interact with different movies
    print('sending event ' + event_type + ' for ' + get_movie_by_id(item_id))
    send_movie_click(USER_ID=user_id, ITEM_ID=item_id, EVENT_TYPE=event_type)
    # Get the recommendations (note you should have a base recommendation DF created before)
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        artist = get_movie_by_id(item['itemId'])
        recommendation_list.append(artist)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [movie_name])
    # Add this dataframe to the old one
    #recommendations_df = recommendations_df.join(new_rec_DF)
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

We will iterate through all of the sammple users and both click on and watch the 4 new titles.

In [124]:
# Note this will take about 20 seconds to complete due to the sleeps
for user in users:
    print ('user ' + str(user))
    for index, row in new_titles_df.iterrows():
        user_id=user
        print('sending event click for ' + get_movie_by_id(index))
        send_movie_click(user_id, index,'click')
        print('sending event watch for ' + get_movie_by_id(index))
        send_movie_click(user_id, index,'watch')
        sleep(2)

user 116
sending event click for New Title 1 (2021)
sending event watch for New Title 1 (2021)
sending event click for New Title 2 (2021)
sending event watch for New Title 2 (2021)
sending event click for New Title 3 (2021)
sending event watch for New Title 3 (2021)
sending event click for New Title 4 (2021)
sending event watch for New Title 4 (2021)
user 471
sending event click for New Title 1 (2021)
sending event watch for New Title 1 (2021)
sending event click for New Title 2 (2021)
sending event watch for New Title 2 (2021)
sending event click for New Title 3 (2021)
sending event watch for New Title 3 (2021)
sending event click for New Title 4 (2021)
sending event watch for New Title 4 (2021)
user 369
sending event click for New Title 1 (2021)
sending event watch for New Title 1 (2021)
sending event click for New Title 2 (2021)
sending event watch for New Title 2 (2021)
sending event click for New Title 3 (2021)
sending event watch for New Title 3 (2021)
sending event click for New

Now lets apply item filters to see recommendations for one of these users within a genre


In [115]:
if not USE_FULL_MOVIELENS:
    users = random.sample(range(1, 600), 3)
else:
    users = random.sample(range(1, 162000), 3)
users

[116, 471, 369]

In [116]:
recommendations_df_users = pd.DataFrame()
#users = users_df.sample(3).index.tolist()

for user in users:
    recommendations_df_users = get_new_recommendations_df_users(recommendations_df_users, user)

recommendations_df_users

,116,471,369
0,"Sound of Music, The (1965)",New Title 1 (2021),Rocky (1976)
1,Shrek 2 (2004),New Title 4 (2021),Amadeus (1984)
2,Singin' in the Rain (1952),New Title 3 (2021),One Flew Over the Cuckoo's Nest (1975)
3,Mulan (1998),Interstellar (2014),"Karate Kid, The (1984)"
4,Doctor Dolittle (1967),New Title 2 (2021),"Natural, The (1984)"
5,Lady and the Tramp (1955),The Nice Guys (2016),Raging Bull (1980)
6,"Hunchback of Notre Dame, The (1996)",Batman Begins (2005),New Title 3 (2021)
7,New Title 1 (2021),Despicable Me (2010),American Graffiti (1973)
8,Robin Hood (1973),Planet Earth II (2016),Pink Floyd: The Wall (1982)
9,Pocahontas (1995),Moana (2016),"Elephant Man, The (1980)"


You can see the recommendations for movies within a given genre. Within a VOD application you could create Shelves (also known as rails or carosels) easily by using these filters. Depending on the information you have about your items, You could also filter on additional information such as keyword, year/decade etc.

## Wrap up <a class="anchor" id="wrapup"></a>
[Back to top](#top)

With that you now have a fully working collection of models to tackle various recommendation and personalization scenarios, as well as the skills to manipulate customer data to better integrate with the service, and a knowledge of how to do all this over APIs and by leveraging open source data science tools.

Use these notebooks as a guide to getting started with your customers for POCs. As you find missing components, or discover new approaches, cut a pull request and provide any additional helpful components that may be missing from this collection.

You'll want to make sure that you clean up all of the resources deployed during this POC. We have provided a separate notebook which shows you how to identify and delete the resources in `07_Clean_Up.ipynb`.